In [167]:
import torch as torch
import pandas as pd
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


# Data Preparation

In [168]:
labels = pd.read_csv('train_label.csv')
labels

,image_index,superclass_index,subclass_index
0,0.jpg,1,53
1,1.jpg,1,45
2,2.jpg,1,4
3,3.jpg,2,7
4,4.jpg,1,33
...,...,...,...
6467,6467.jpg,0,55
6468,6468.jpg,0,84
6469,6469.jpg,0,49
6470,6470.jpg,2,71


In [169]:
# subclass labels of each super class
class0 = labels[labels['superclass_index']==0]
class1 = labels[labels['superclass_index']==1]
class2 = labels[labels['superclass_index']==2]
sub_class0 = class0.subclass_index.unique()
sub_class1 = class1.subclass_index.unique()
sub_class2 = class2.subclass_index.unique()
#labels.subclass_index.value_counts()

In [170]:
# make a image folder which contain labels
'''
import os
import shutil
train_path = 'train_shuffle/'
cleaned_ds = 'cleaned_train_dataset/'
for i in range(len(labels)):
    image_path = os.path.join(train_path, labels['image_index'][i])
    new_path = cleaned_ds+str(labels['superclass_index'][i])+'/'+str(labels['subclass_index'][i])+'/'
    new_image = cleaned_ds+str(labels['superclass_index'][i])+'/'+str(labels['subclass_index'][i])+'/'+labels['image_index'][i]
    if os.path.exists(new_path) == True:
        shutil.copyfile(image_path, new_image)
    else:
        os.mkdir(new_path)
        shutil.copyfile(image_path, new_image)
'''
        

"\nimport os\nimport shutil\ntrain_path = 'train_shuffle/'\ncleaned_ds = 'cleaned_train_dataset/'\nfor i in range(len(labels)):\n    image_path = os.path.join(train_path, labels['image_index'][i])\n    new_path = cleaned_ds+str(labels['superclass_index'][i])+'/'+str(labels['subclass_index'][i])+'/'\n    new_image = cleaned_ds+str(labels['superclass_index'][i])+'/'+str(labels['subclass_index'][i])+'/'+labels['image_index'][i]\n    if os.path.exists(new_path) == True:\n        shutil.copyfile(image_path, new_image)\n    else:\n        os.mkdir(new_path)\n        shutil.copyfile(image_path, new_image)\n"

### calculate mean and std of images

In [171]:
def get_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0
    for images, _ in loader:
        images = images.view(images.size(0), images.size(1),-1)
        mean +=images.mean(2).sum(0)
        std +=images.std(2).sum(0)
        total_images_count += images.size(0)
    mean /= total_images_count
    std /= total_images_count
    return mean, std
training_transforms = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.ImageFolder(root='super-sub-data/super-sub', 
                                                 transform = training_transforms)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size=64, shuffle=False)

In [172]:
print(std)

tensor([0.6136, 0.6157, 0.6193])


In [173]:
def set_device():
    if torch.cuda.is_available():
        dev = 'cuda:0'
    else:
        dev = 'cpu'
    return torch.device(dev)

In [174]:
import os
folder_path = 'super-sub-data/super-sub/'
folder_contents = os.listdir(folder_path)

# Count the number of subfolders in the folder
num_subfolders = sum([os.path.isdir(os.path.join(folder_path, item)) for item in folder_contents])

# Print the result
print(f'There are {num_subfolders} subfolders in the folder.')

There are 3 subfolders in the folder.


### transform data

In [175]:
from torch.utils.data import Dataset
from torchvision import models
test_path = 'test_shuffle/'
train_path = 'super-sub-data/super-sub/'
####################tbd
mean, std = get_mean_std(train_loader)
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomAutocontrast(p=0.5),
    #transforms.RandomAdjustSharpness(sharpness_factor=2,p=0.5),
    transforms.ColorJitter()
    #transforms.RandomEqualize()
    #transforms.AugMix(),
    #transforms.ToTensor(),
    #transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    
])
#randaugment_transforms = transforms.Compose([transforms.RandAugment()])
test_transforms = transforms.Compose([
    transforms.ToTensor()
    #transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    
])
class DatasetFromSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)
dataset = torchvision.datasets.ImageFolder(root = train_path, transform = test_transforms)
#build my own class
#


#dataset = torch.utils.data.DataLoader(dataset=dataset, shuffle=True)

# n = len(dataset)  # total number of examples
# n_test = int(0.13 * n)  # take ~15% for test
# test_temp_set = torch.utils.data.Subset(dataset, range(0,n_test))
# dev_set = torch.utils.data.Subset(dataset, range(n_test, n))  # take the rest

#split data into dev and test
len_dev = int(len(dataset)*0.85)
len_testtemp = len(dataset)-len_dev
test_temp_set, dev_set = torch.utils.data.random_split(dataset, [len_testtemp, len_dev])

dataset_duplicate_1 = DatasetFromSubset(dev_set, transform = train_transforms)
dataset_duplicate_2 = DatasetFromSubset(dev_set, transform = train_transforms)
dataset_duplicate_3 = DatasetFromSubset(dev_set, transform = train_transforms)

#dataset_duplicate_1 = DatasetFromSubset(dev_set, transform = RandAugment_transforms)

dev_set = dev_set + dataset_duplicate_1 + dataset_duplicate_2 + dataset_duplicate_3




In [176]:
# dev_loader = torch.utils.data.DataLoader(dataset = dev_set, batch_size=32, shuffle=True)


#split dev into val and train
train_len = int(len(dev_set)*0.85)
val_len = len(dev_set)-train_len
train_loader, val_loader = torch.utils.data.random_split(dev_set, [train_len, val_len])
train_loader = torch.utils.data.DataLoader(train_loader, batch_size=64)
val_loader = torch.utils.data.DataLoader(val_loader, batch_size=64)

# Train Model

In [177]:
#resnet
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

model = models.wide_resnet50_2(pretrained=False)
num_features = model.fc.in_features
number_classes=3
model.fc = nn.Linear(num_features, number_classes)
device = set_device()
print(device)
model = model.to(device)
loss_function = nn.CrossEntropyLoss()
#optimizer = optim = optim.Adam(resnet50_model.parameters(),lr=0.01)

cuda:0


In [178]:
#alexnet
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim


In [179]:
#use learning rate scheduler to train the model
from torch.optim.lr_scheduler import ReduceLROnPlateau
def evaluate_model_on_test_set(model, test_loader):
    model.eval()
    predicted_correctly_on_epoch = 0
    total = 0
    device = set_device()
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            total+= labels.size(0)
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100*predicted_correctly_on_epoch/total
    print('testing dataset: got %d out of %d images. Accuracy(%.3f%%),'
             %(predicted_correctly_on_epoch, total, epoch_acc))
    return epoch_acc

# Define the learning rate schedule function
def lr_schedule(epoch):
#     if epoch < 10:
#         return 0.02
#     elif epoch < 20:
#         return 0.01
#     else:
#         return 0.001
    return 0.01

def train_nn(model, train_loader, test_loader, criterion, epochs):
    validation_accuracy_array = [0,0,0,0,0,0,0,0,0,0]
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
    scheduler = ReduceLROnPlateau(optimizer, factor = 0.5, patience=6,verbose=True)
    device = set_device()
    for epoch in range(epochs):        
        print('epoch: ',epoch)
        model.train()
        running_loss=0
        running_correct=0
        total=0
        for data in train_loader:

            images, labels = data

            images = images.to(device)
            labels = labels.to(device)
            total+= labels.size(0)
            #plt.imshow(images)
            optimizer.zero_grad()
            outputs = model(images)
            _, predicted = torch.max(outputs.data,1)
#             print(outputs.data.shape)
#             print(predicted)
#             print()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()
        print(optimizer.param_groups[0]['lr'])
        epoch_loss = running_loss/len(train_loader)
        epoch_accuracy = 100*running_correct/total
        print('training dataset: got %d out of %d images. Accuracy(%.3f%%), epoch loss %.3f'
             %(running_correct, total, epoch_accuracy, epoch_loss))
        validation_accuracy = evaluate_model_on_test_set(model, test_loader)
        scheduler.step(validation_accuracy)
        if validation_accuracy > min(validation_accuracy_array):
            validation_accuracy_array[epoch%10] = validation_accuracy
        else:
            print("overfitting.(the updated validation accuracy is less than the minimum validation accuracies of last 10 epochs)")
            break
    return model


train_nn(model, train_loader, val_loader, loss_function, 150)
# add early stop

epoch:  0
0.01
training dataset: got 7852 out of 18842 images. Accuracy(41.673%), epoch loss 2.009
testing dataset: got 1503 out of 3326 images. Accuracy(45.189%),
epoch:  1
0.01
training dataset: got 8445 out of 18842 images. Accuracy(44.820%), epoch loss 1.130
testing dataset: got 1554 out of 3326 images. Accuracy(46.723%),
epoch:  2
0.01
training dataset: got 8767 out of 18842 images. Accuracy(46.529%), epoch loss 1.054
testing dataset: got 1600 out of 3326 images. Accuracy(48.106%),
epoch:  3
0.01
training dataset: got 9534 out of 18842 images. Accuracy(50.600%), epoch loss 1.007
testing dataset: got 1787 out of 3326 images. Accuracy(53.728%),
epoch:  4
0.01
training dataset: got 9977 out of 18842 images. Accuracy(52.951%), epoch loss 0.973
testing dataset: got 1727 out of 3326 images. Accuracy(51.924%),
epoch:  5
0.01
training dataset: got 10244 out of 18842 images. Accuracy(54.368%), epoch loss 0.949
testing dataset: got 1878 out of 3326 images. Accuracy(56.464%),
epoch:  6
0.01


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [180]:
test_temp_loader = torch.utils.data.DataLoader(dataset = test_temp_set, batch_size=64, shuffle=True)
model.eval()
device = set_device()
# Use the model to predict the outputs for the test data
predicted_outputs = []
true_outputs = []
for input_data, target_data in test_temp_loader:
    input_data = input_data.to(device)
    target_data = target_data.to(device)
    # Feed the input data into the model and compute the predicted outputs
    output = model(input_data)
    _, predicted = torch.max(output.data,1)
    # Store the predicted and true outputs
    predicted_outputs.append(predicted)
    true_outputs.append(target_data)
predicted_outputs = torch.cat(predicted_outputs)
true_outputs = torch.cat(true_outputs)
accuracy = torch.mean((predicted_outputs == true_outputs).to(torch.float))
print(accuracy)

tensor(0.7720, device='cuda:0')


In [166]:
mean, std

(tensor([0.2297, 0.2273, 0.2195]), tensor([0.6136, 0.6157, 0.6193]))

In [181]:
import pickle

# Save the model to a file
with open('super_wrs_1.pkl', 'wb') as f:
  pickle.dump(model, f)

In [73]:
import statistics
resnet18 = [0.7642,0.7693, 0.7580]
resnet18 = statistics.mean(resnet18)
resnet34 = [0.7302,0.6869, 0.7683]
resnet34 = statistics.mean(resnet34)
resnet50 = [0.7065,0.5345, 0.4573]
resnet50 = statistics.mean(resnet50)
resnet101 = [0.3996,0.4861, 0.6622]
resnet101 = statistics.mean(resnet101)
resnet152 = [0.5942, 0.6262,0.4748]
resnet152 = statistics.mean(resnet152)

wideresnet50 = 
#models = ['ResNet18','ResNet34','ResNet50', 'ResNet101','ResNet152']
resnet152

0.5650666666666666

resnet50:
lr = 0.01 epoch 20 test accuracy 70.44% 
dynamic lr 10/20 0.02/0.01/0.001 epoch 30 test accuracy 75
lr = 0.01 epoch 30 test accuracy 77.86% 

resnet 152:
 test accuracy 68.59%
resnext101_32x8d
dynamic lr 10/20 0.02/0.01/0.001 epoch 30 test accuracy 75.39%
classify subcategory: 
lr 0.01 epoch 20 without doing contrast test accuracy 9.37%
lr 0.01 epoch 20 with contrast test accuracy 12.87%
lr 0.01 epoch 200 with contrast, test accuracy 21.06%/10.6%

wide_resnet50_2:
30 epochs 71.88%
0.74.56% with auto contrast
0.7353 with auto contrast and RandomAdjustSharpness
77.96 epoch 62

wide_resnet101_2
lr 0.01 epoch 100, test accuracy 22.19%/14.1%
lr 0.01 epoch 100 with ReduceLROnPlateau accuracy 21.987/13.6

In [ ]:
# def get_super_class_pred(predicted_outputs):
#     super_class = []
#     for i in predicted_outputs:
#         if i.item() in sub_class0:
#             super_class.append(0)
#         elif i.item() in sub_class1:
#             super_class.append(1)
#         elif i.item() in sub_class2:
#             super_class.append(2)
#     return super_class

# def get_similarity(arr1, arr2):
#     # Calculate the number of elements that are the same in both arrays
#     same_count = sum(1 for a, b in zip(arr1, arr2) if a == b)

#     # Calculate the total number of elements in both arrays
#     total_count = len(arr1) + len(arr2)

#     # Calculate the similarity percentage
#     similarity = same_count / total_count

#     return similarity
# # Evaluate the model using test-temp data
# test_temp_loader = torch.utils.data.DataLoader(dataset = test_temp_set, batch_size=64, shuffle=True)
# model.eval()
# device = set_device()
# # Use the model to predict the outputs for the test data
# predicted_sub_outputs = []
# true_sub_outputs = []
# for input_data, target_data in test_temp_loader:
#     input_data = input_data.to(device)
#     target_data = target_data.to(device)
#     # Feed the input data into the model and compute the predicted outputs
#     output = model(input_data)
#     _, predicted = torch.max(output.data,1)
#     # Store the predicted and true outputs
#     predicted_sub_outputs.append(predicted)
#     true_sub_outputs.append(target_data)

# predicted_sub_outputs = torch.cat(predicted_sub_outputs)
# true_sub_outputs = torch.cat(true_sub_outputs)
# true_super_outputs = get_super_class_pred(true_sub_outputs)
# sub_accuracy = torch.mean((predicted_sub_outputs == true_sub_outputs).to(torch.float))


# predicted_super_outputs = get_super_class_pred(predicted_sub_outputs)

# super_accuracy = get_similarity(predicted_super_outputs,true_super_outputs)*100


# print('super class accuracy', super_accuracy)
# print("sub class accuracy",sub_accuracy.item())

